<a href="https://colab.research.google.com/github/DrBryne/colab/blob/main/Code_Interpreter_Extension_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test for your Vertex Code Interpreter Extension
---

## Install necessary dependencies

In [1]:
!pip install google-cloud-aiplatform==1.29.0

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 397, in resolve
    self._add_to_criteria(self.state.criteria, r, parent

## Authenticate and Set Up Environment

This notebook requires that you have the following permissions for your GCP project:
* `roles/aiplatform.user`

The following APIs must be enabled:
* Vertex AI API

The project must also be allowlisted for Model Extend with the `EXTENSION_TRUSTED_TESTER2` label.

In [ ]:
PROJECT_ID = "model-extend-test"  # @param ["model-extend-test"]


In [ ]:
from google.colab import auth

auth.authenticate_user(project_id=PROJECT_ID)

# Application Default Credentials are necessary to get an identity token to call
# Vertex APIs.
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  Y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=fMRk2nZMkuHYsAWCiqCvon4U1UdKp1&prompt=consent&access_type=offline&code_challenge=l5iKd

### Execute the Code Interpreter Extension



#### Set Up Input Files


> The file contents need to be **base64-encoded** and be passed in the the following structure:


```
[
  {
    "name": "data.csv",
    "contents": "aXRlbTEsaXRlbTI="
  }
]
```

In [ ]:
import base64

# Update this to upload files
# INPUT_FILES_PATH = []
INPUT_FILES_PATH = ["./sample_data/california_housing_test.csv"]


# Helper method to read and encode files.
def upload_file(file_paths):
    files = []
    for file_path in file_paths:
      with open(file_path, "rb") as file:
        encoded_string = base64.b64encode(file.read()).decode()
        files.append({
            "name": file_path.split("/")[-1], "contents": encoded_string})
    return files


FILES = upload_file(INPUT_FILES_PATH)
print("Input Files:", [f["name"] for f in FILES])

Input Files: ['california_housing_test.csv']


#### Set Up Query

In [ ]:
# Type 1: Prompt using data from the input files from `FILES` above.
QUERY="From the attached CSV file, can you tell me which house has the maximum of median house value? Keep only the first row."

# Type 2: Prompt with simple instructions.
# QUERY = "find the first fibonacci number greater than 999."

# Type 3: Prompt with data directly embedded.
# QUERY = "Using the data below, construct pie chart to represent the heights only. \nflower_heights_prices = {\n    \"Tulip\": {\"height\": 20, \"price\": 35},\n    \"Orchid\": {\"height\": 120, \"price\": 50},\n    \"Sunflower\": {\"height\": 100, \"price\": 34},\n    \"Daisy\": {\"height\": 30, \"price\": 5},\n    \"Rose\": {\"height\": 40, \"price\": 33},\n    \"Carnation\": {\"height\": 25, \"price\": 9},\n    \"Hyacinth\": {\"height\": 20, \"price\": 11},\n    \"Iris\": {\"height\": 70, \"price\": 47},\n    \"Lily\": {\"height\": 80, \"price\": 12},\n    \"Jasmine\": {\"height\": 30, \"price\": 13},\n} use python code."

print("Query:", QUERY)

Query: From the attached CSV file, can you tell me which house has the maximum of median house value? Keep only the first row.


#### Execute Extension and Collect Results



In [ ]:
import json
import requests

token = !gcloud auth application-default print-access-token

ENDPOINT = "https://us-central1-autopush-aiplatform.sandbox.googleapis.com"
EXTENSION_ID = 50243283342852096
OPERATION_ID = "generate_and_execute"

response = requests.post(
    f"{ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/us-central1/extensions/{EXTENSION_ID}:execute",
    headers={
        "Content-Type": "application/json; charset=utf-8",
        "Authorization": f"Bearer {token[0]}"
    },
    data=json.dumps({
        "operation_id": OPERATION_ID,
        "operation_params": {
          "query": QUERY,
          "files": FILES
        }
    })
)

if response.status_code != 200:
    raise Exception(f"ExecuteExtension Failed: {response.content}")
else:
  print("ExtensionExecution Succeeded.")

ExtensionExecution Succeeded.


In [ ]:
import base64
import io
import pprint
from PIL import Image


IMAGE_FILE_EXTENSIONS = set(["jpg", "jpeg", "png"])

if response.status_code != 200:
    raise Exception(f"Failed to execute extension: {response.content}")
else:
  jsonResponse = json.loads(json.loads(response.content).get("output").get("content"))
  print(">> ExtensionExecution Response: \n==============================================\n")
  print("Generated Code: \n=======================\n")
  print(jsonResponse.get("generated_code") + "\n\n")
  print("Code Execution Result: \n=======================\n")
  print("\"" + jsonResponse.get("execution_result") + "\"\n\n")
  print("Code Execution Error: \n=======================\n")
  print("\"" + jsonResponse.get("execution_error") + "\"\n\n")

# Sort the output_files so images are displayed before other files such as json.
for output_file in sorted(jsonResponse.get("output_files", []), key=lambda x: x["name"].split(".")[-1] not in IMAGE_FILE_EXTENSIONS):
    file_name = output_file.get("name")
    file_contents = base64.b64decode(output_file.get("contents"))
    print("Output Files: \n=======================\n")
    print(f"File Name: {file_name}\n")

    # Render Image.
    if file_name.split(".")[-1] in IMAGE_FILE_EXTENSIONS:
      image = Image.open(io.BytesIO(file_contents))
      display(image)
    # Pretty print Json.
    elif file_name.endswith(".json"):
      pprint.pprint(json.loads(file_contents.decode()), compact=False, width=160)
    else:
      print(f"File Contents: {file_contents.decode()}\n")

>> ExtensionExecution Response: 

Generated Code: 

```python
import pandas as pd

# Read the CSV file
df = pd.read_csv("california_housing_test.csv")

# Find the house with the maximum median house value
max_house_value = df["median_house_value"].max()
house_with_max_value = df[df["median_house_value"] == max_house_value]

# Keep only the first row
house_with_max_value = house_with_max_value.head(1)

# Print the result
print(house_with_max_value)
```


Code Execution Result: 

"    longitude  latitude  housing_median_age  total_rooms  total_bedrooms  population  households  median_income  median_house_value
42 -118.06     34.15     37.0                1980.0       226.0           697.0       226.0       15.0001        500001.0          
"


Code Execution Error: 

""


